# Data Preprocessing
The dataset provided for this challenge is composed by xx images of different height and width. In order to obtain a standard shape for all images, a particular function has been used, namely `tf.keras.preprocessing.image.smart_resize()`.

This function resizes images to a specified target size without affecting the aspect ratio. This step is due to the successive batching of all images that requires a common format for all of them.
In order to avoid a loss of information that would have been caused by applying a crop, it has been computed what the max width/height of the images was by implementing `getMaxImageSize(dataset_dir)`.

This function returns the maximum (width,height), considering all the images of the provided dataset.
For what regards data augmentation and the splitting of the data into two distinct sets, namely train and validation, it has been used the following script which assigns a fraction of 30% of the imgaes to the validation set; the rest will be assigned to the training set.

```
train_data_gen = ImageDataGenerator ( rotation_range=10,            
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3, 
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      rescale=1. / 255,
                                      validation_split=0.3,  
                                      preproc_funct=smart_resize()  
                                    )
```
Considering that a strict division of the images in subdirectories representing the target classes is required for the functions involved in the creation of augmented images, we implemented a specific function that requires a ```json_definition``` for the subdirectory construction and the path where such images are located:

```
divideDatasetInTargetFolders(json_definition, dataset_path)
```

# Model Design
# Model Selection
#Results





